In [1]:
from bs4 import BeautifulSoup
import requests as re
import pandas as pd
import numpy as np
import os

In [2]:
pd.set_option('display.max_colwidth', -1)

In [3]:
%%time
main_table = []
#url = input("http://www.e-stat.go.jp/SG1/estat/OtherListE.do?bid=000001006005&cycode=1")
url = "http://www.e-stat.go.jp/SG1/estat/OtherListE.do?bid=000001006005&cycode=1"
path='https://www.e-stat.go.jp'
r = re.get(url)
data = r.text
year='None'
soup = BeautifulSoup(data, "lxml")
table = soup.find('div', {'class': 'stat-cycle_sheet'})
for year_section in table.find_all('ul', {'class': 'stat-cycle_ul_other'}):
    header = year_section.find('li', {'class': 'stat-cycle_header'})
    year = header.find('span').get_text(' ', strip=True)
    #   print('year changed to ' + year)
    months_section = year_section.find('li', {'class': 'stat-cycle_item'})
    for month_row in months_section.find_all('div'):
        month_a = month_row.find('a')
        month = month_a.get_text().rstrip('.\n')
        monthurl = path + month_a.get('href')
        
        main_table.append({
            'year': year,
            'month': month,
            'url': monthurl
        })

top_df = pd.DataFrame(main_table)

Wall time: 2.12 s


Retrieve available months by parsing website

In [4]:
def retrieve_month_excels(murl, path):
    print(murl)
    month_table = []
#     mcols = ['url', 'excel_num', 'excel_description', 'excel_url']
#     mdf = pd.DataFrame(columns=mcols)
    mr = re.get(murl)
    mdata = mr.text
    msoup = BeautifulSoup(mdata, "lxml")
    mtable = msoup.find('div', {'class': 'stat-dataset_list-body'})
    for row in mtable.find_all('article', {'class': 'stat-dataset_list-item'}):
        excel_num = row.find('li', {'class': 'stat-dataset_list-detail-item stat-dataset_list-border-top'}).contents[0].replace('\n','')
        excel_description = row.find('a').contents[0]
        excel_url = ''
        excel_a = row.find_all('a')[1]
        
        if(excel_a['data-file_type'] == 'EXCEL'):
            excel_url = path + excel_a['href']
            
        month_table.append({
            'url': murl,
            'excel_num': excel_num,
            'excel_description': excel_description,
            'excel_url': excel_url
        })
#         mdfrow = pd.DataFrame([[murl, excel_num, excel_description, excel_url]], columns=mcols)
#         if(len(mdf)==0):
#             mdf = mdfrow
#         else:
#             mdf = mdf.append(mdfrow, ignore_index=True) #why the hell doesn't df.append work inplace?? Didn't it always use to?
    mdf = pd.DataFrame(month_table)
    return(mdf)

Define function to enter link for each month, and store the excel notebook links and descriptions within to a dataframe.

In [5]:
%%time
excels_df = pd.concat([retrieve_month_excels(murl, path) for murl in top_df['url']])
excelref_df = top_df.merge(excels_df, how='left', on='url')

https://www.e-stat.go.jp/en/stat-search/files?cycle=1&layout=datalist&toukei=00200523&tstat=000000070001&tclass1=000001008739&year=20180&month=11010301&result_back=1
https://www.e-stat.go.jp/en/stat-search/files?cycle=1&layout=datalist&toukei=00200523&tstat=000000070001&tclass1=000001008739&year=20180&month=11010302&result_back=1
https://www.e-stat.go.jp/en/stat-search/files?cycle=1&layout=datalist&toukei=00200523&tstat=000000070001&tclass1=000001008739&year=20180&month=11010303&result_back=1
https://www.e-stat.go.jp/en/stat-search/files?cycle=1&layout=datalist&toukei=00200523&tstat=000000070001&tclass1=000001008739&year=20180&month=12040604&result_back=1
https://www.e-stat.go.jp/en/stat-search/files?cycle=1&layout=datalist&toukei=00200523&tstat=000000070001&tclass1=000001008739&year=20180&month=12040605&result_back=1
https://www.e-stat.go.jp/en/stat-search/files?cycle=1&layout=datalist&toukei=00200523&tstat=000000070001&tclass1=000001008739&year=20180&month=12040606&result_back=1
http

https://www.e-stat.go.jp/en/stat-search/files?cycle=1&layout=datalist&toukei=00200523&tstat=000000070001&tclass1=000001008739&year=20140&month=23070909&result_back=1
https://www.e-stat.go.jp/en/stat-search/files?cycle=1&layout=datalist&toukei=00200523&tstat=000000070001&tclass1=000001008739&year=20140&month=24101210&result_back=1
https://www.e-stat.go.jp/en/stat-search/files?cycle=1&layout=datalist&toukei=00200523&tstat=000000070001&tclass1=000001008739&year=20140&month=24101211&result_back=1
https://www.e-stat.go.jp/en/stat-search/files?cycle=1&layout=datalist&toukei=00200523&tstat=000000070001&tclass1=000001008739&year=20140&month=24101212&result_back=1
https://www.e-stat.go.jp/en/stat-search/files?cycle=1&layout=datalist&toukei=00200523&tstat=000000070001&tclass1=000001008739&year=20130&month=11010301&result_back=1
https://www.e-stat.go.jp/en/stat-search/files?cycle=1&layout=datalist&toukei=00200523&tstat=000000070001&tclass1=000001008739&year=20130&month=11010302&result_back=1
http

https://www.e-stat.go.jp/en/stat-search/files?cycle=1&layout=datalist&toukei=00200523&tstat=000000070001&tclass1=000001008739&year=20100&month=24101211&result_back=1
https://www.e-stat.go.jp/en/stat-search/files?cycle=1&layout=datalist&toukei=00200523&tstat=000000070001&tclass1=000001008739&year=20100&month=24101212&result_back=1
https://www.e-stat.go.jp/en/stat-search/files?cycle=1&layout=datalist&toukei=00200523&tstat=000000070001&tclass1=000001008739&year=20090&month=11010301&result_back=1
https://www.e-stat.go.jp/en/stat-search/files?cycle=1&layout=datalist&toukei=00200523&tstat=000000070001&tclass1=000001008739&year=20090&month=11010302&result_back=1
https://www.e-stat.go.jp/en/stat-search/files?cycle=1&layout=datalist&toukei=00200523&tstat=000000070001&tclass1=000001008739&year=20090&month=11010303&result_back=1
https://www.e-stat.go.jp/en/stat-search/files?cycle=1&layout=datalist&toukei=00200523&tstat=000000070001&tclass1=000001008739&year=20090&month=12040604&result_back=1
http

Wall time: 3min 46s


In [6]:
excelref_df.head()

,month,url,year,excel_description,excel_num,excel_url
0,Jan,https://www.e-stat.go.jp/en/stat-search/files?cycle=1&layout=datalist&toukei=00200523&tstat=000000070001&tclass1=000001008739&year=20180&month=11010301&result_back=1,2018,"Number of Intra-prefectural Migrants, In-migrants from Other Prefectures and Out-migrants to Other Prefectures by Sex for Japan, Prefectures, Tokyo Area, Nagoya Area, Osaka Area and 21 Major Cities (All nationalities and Japanese)",1,https://www.e-stat.go.jp/en/stat-search/file-download?statInfId=000031673390&fileKind=0
1,Jan,https://www.e-stat.go.jp/en/stat-search/files?cycle=1&layout=datalist&toukei=00200523&tstat=000000070001&tclass1=000001008739&year=20180&month=11010301&result_back=1,2018,"Number of Inter-prefectural Migrants by Sex and Origin for Prefectures, Tokyo Area, Nagoya Area, Osaka Area and 21 Major Cities (Japanese)",2,https://www.e-stat.go.jp/en/stat-search/file-download?statInfId=000031673391&fileKind=0
2,Jan,https://www.e-stat.go.jp/en/stat-search/files?cycle=1&layout=datalist&toukei=00200523&tstat=000000070001&tclass1=000001008739&year=20180&month=11010301&result_back=1,2018,Number of In-migrants from Other Prefectures by Age (Five-Year Groups) and Sex for Japan and Prefectures (Japanese),3-1,https://www.e-stat.go.jp/en/stat-search/file-download?statInfId=000031673392&fileKind=0
3,Jan,https://www.e-stat.go.jp/en/stat-search/files?cycle=1&layout=datalist&toukei=00200523&tstat=000000070001&tclass1=000001008739&year=20180&month=11010301&result_back=1,2018,Number of Out-migrants to Other Prefectures by Age (Five-Year Groups) and Sex for Japan and Prefectures (Japanese),3-2,https://www.e-stat.go.jp/en/stat-search/file-download?statInfId=000031673393&fileKind=0
4,Jan,https://www.e-stat.go.jp/en/stat-search/files?cycle=1&layout=datalist&toukei=00200523&tstat=000000070001&tclass1=000001008739&year=20180&month=11010301&result_back=1,2018,Number of Net-migration by Age (Five-Year Groups) and Sex for Japan and Prefectures (All nationalities and Japanese),3-3,https://www.e-stat.go.jp/en/stat-search/file-download?statInfId=000031673394&fileKind=0


Utilize the function retrieve_month_excels to retrieve all available excel sheets for every month in the top-level dataframe

In [7]:
exceldesccount_df = excelref_df.groupby(['excel_description']).agg(['count']).sort_values([('year', 'count')], ascending=False).iloc[:,:1]
exceldesccount_df.columns=['count']
# exceldesccount_df.insert(0,'excel_description', exceldesccount_df.index)
exceldesccount_df.reset_index(inplace=True)
print(exceldesccount_df)

                                                                                                                                                                                                                            excel_description  \
0   
Number of In-migrants from Other Prefectures by Age (5-Year Age Group) and Sex for Japan and Prefectures
                                                                                                                                  
1   
Number of Out-migrants to Other Prefectures by Age (5-Year Age Group) and Sex for Japan and Prefectures
                                                                                                                                   
2   
Number of Net-migration by Age (5-Year Age Group) and Sex for Japan and Prefectures (All nationalities and Japanese)
                                                                                                                      
3   
Number of Inter-prefectural Mig

Count of sheets of each description in all available data

In [8]:
%%time
# print(excelref_df.head())
mdir = os.path.join('extracts', 'monthly_reports')
if not os.path.exists(mdir):
    os.makedirs(mdir)
months = excelref_df.month.unique() #do not sort, retain the order
monthnum = list(range(1,13,1))
monthdirs = [str(num).zfill(2) + month for num,month in zip(monthnum, months)]
monthnamedict = dict(zip(months, monthdirs))
print(monthnamedict)

for index,row in excelref_df.iterrows():
    filedir = os.path.join(mdir, row['year'], monthnamedict.get(row['month']))
    if not os.path.exists(filedir):
        os.makedirs(filedir)
    filename = str(row['excel_num']) + '.xls'
    savepath = os.path.join(filedir, filename)
    if not os.path.isfile(savepath):
        fileresponse = re.get(row['excel_url'], stream=True)
        with open(savepath, 'wb') as out_file:
            out_file.write(fileresponse.content)
        del fileresponse
        print("saved " + savepath)

{'Jan': '01Jan', 'Feb': '02Feb', 'Mar': '03Mar', 'Apr': '04Apr', 'May': '05May', 'Jun': '06Jun', 'Jul': '07Jul', 'Aug': '08Aug', 'Sep': '09Sep', 'Oct': '10Oct', 'Nov': '11Nov', 'Dec': '12Dec'}
Wall time: 165 ms


Save all files available to disk if they don't already exist

In [9]:
samplecols = ['sampleyear', 'samplemonth', 'sampleexcel_num', 'excel_description', 'count']
samplebydesc_df = pd.DataFrame(columns=samplecols)
samplebydesc_df['excel_description'] = exceldesccount_df['excel_description']
samplebydesc_df['count'] = exceldesccount_df['count']

I want to build a dataframe of example excel workbooks of each description, so that i can formulate a way to parse each workbook according to their description.

In [10]:
# sampledesc = 'Number of In-migrants from Other Prefectures by Age (5-Year Age Group) and Sex for Japan and Prefectures'
# dfdf1 = excelref_df[excelref_df['excel_description'] == sampledesc][:1]['month'].iloc[0]
samplebydesc_df['sampleyear'] = [excelref_df[excelref_df['excel_description'] == x][:1]['year'].iloc[0]\
                                 for x in samplebydesc_df['excel_description']]
samplebydesc_df['samplemonth'] = [excelref_df[excelref_df['excel_description'] == x][:1]['month'].iloc[0]\
                                 for x in samplebydesc_df['excel_description']]
samplebydesc_df['sampleexcel_num'] = [excelref_df[excelref_df['excel_description'] == x][:1]['excel_num'].iloc[0]\
                                 for x in samplebydesc_df['excel_description']]
print(samplebydesc_df[:20])

   sampleyear samplemonth sampleexcel_num  \
0   2014       Jan         3-1              
1   2014       Jan         3-2              
2   2017       Jan         3-3              
3   2018       Jan         2                
4   2013       Jan         3-3              
5   2017       Jan         1                
6   2017       Jan         3-2              
7   2017       Jan         3-1              
8   2014       Jan         2                
9   2009       Jan         2                
10  2009       Jan         1                
11  2012       Jan         2                
12  2012       Jan         1                
13  2013       Jan         1                
14  2014       Jan         1                
15  2010       Jan         1                
16  2006       Jan         1                
17  2006       Jan         2                
18  2009       Apr         2                
19  2018       Jan         1                

                                                      

The first excel workbook with the matching description is assigned as the sample workbook